In [ ]:
import pandas as pd

from get_game_information import team_info, fetch_games, get_game_info
from game_simulator import outcomes, calculate_total_bases, simulator, create_detailed_outcomes_df, outcome_rankings
from visualizations import run_dist, la_ev_graph, create_estimated_bases_table
from constants import  mlb_team_logos

## See teams below
teams_df = team_info()

## How far back you want to go for games
days_ago = 1.25

games_df, games_list = fetch_games(days_ago, all_columns = False)

## Add short team names
games_df = pd.merge(games_df, teams_df[['team.id', 'teamName']], left_on='teams.away.team.id', right_on='team.id')
games_df = games_df.rename(columns={'teamName': 'away.team'})
games_df = pd.merge(games_df, teams_df[['team.id', 'teamName']], left_on='teams.home.team.id', right_on='team.id')
games_df = games_df.rename(columns={'teamName': 'home.team'})
games_df = games_df.drop(columns=['team.id_x', 'team.id_y'])
games_df = games_df.dropna(subset=['teams.home.score']).reset_index(drop=True)

## Loop through games
for game_id in games_list:
  ## Gather hit data and all data (all_bb) for sb, cs, po, errors
  game_data, all_bb = get_game_info(game_id, all_columns = False)
  if game_data is None:
      print(f"Error retrieving data for game ID: {game_id}")
      continue
  ## venue equals value from game_data where gamePk == game_id
  venue = games_df[games_df['gamePk'] == game_id]['venue.name'].values[0]
  game_data['venue.name'] = venue
  ## Get actual game info
  game_info = games_df.copy()
  game_info = game_info[game_info['gamePk'] == game_id]
  if len(game_info) == 0:
      print(f"No game found for game ID: {game_id}")
      continue
  home_team = game_info['home.team'].values[0]
  away_team = game_info['away.team'].values[0]
  home_score = int(game_info['teams.home.score'].iloc[0])
  away_score = int(game_info['teams.away.score'].iloc[0])
  print(f"{away_team} @ {home_team}")

  home_outcomes = outcomes(game_data, 'home')
  away_outcomes = outcomes(game_data, 'away')

  # Calculate the estimated total bases
  home_estimated_total_bases = calculate_total_bases(home_outcomes)
  away_estimated_total_bases = calculate_total_bases(away_outcomes)

  # For home team
  home_detailed_df = create_detailed_outcomes_df(game_data, 'home')
  home_detailed_df['team'] = home_team
  # For away team
  away_detailed_df = create_detailed_outcomes_df(game_data, 'away')
  away_detailed_df['team'] = away_team

  ## Get dataframes for estimated bases graphs
  top_10_bases = outcome_rankings(home_detailed_df, away_detailed_df)

  ## Clean up the tuples
  home_outcomes = [outcome[0] if isinstance(outcome[0], tuple) else outcome[0] for outcome in home_outcomes]
  away_outcomes = [outcome[0] if isinstance(outcome[0], tuple) else outcome[0] for outcome in away_outcomes]

  num_simulations = 2000
  home_runs_scored, away_runs_scored, home_win_percentage, away_win_percentage, tie_percentage = simulator(num_simulations, home_outcomes, away_outcomes)
  la_ev_graph(home_outcomes, away_outcomes, home_runs_scored, away_runs_scored, home_team, away_team, home_score, away_score, home_win_percentage, away_win_percentage, tie_percentage, mlb_team_logos)
  run_dist(num_simulations, home_runs_scored, away_runs_scored, home_team, away_team, home_score, away_score, home_win_percentage, away_win_percentage, tie_percentage)
  create_estimated_bases_table(top_10_bases, away_team, home_team, away_score, home_score, away_win_percentage, home_win_percentage)
